In [1]:
#the torch init

import torch

In [2]:
#libraries

import numpy 
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import torch.nn as  nn


In [3]:
#linear reg using tensors and autograd
#it apparently worked here maybe cuz i updated wsl
housing = fetch_california_housing()


In [4]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [5]:
housing.target_names

['MedHouseVal']

In [6]:
x_train_full , x_test , y_train_full , y_test = train_test_split(housing.data , housing.target ,
                                                                 test_size=0.15,random_state=42)

In [7]:
type(x_train_full)

numpy.ndarray

In [8]:
y_train_full.shape

(17544,)

In [9]:
y_train_full.shape

(17544,)

In [10]:
x_train , x_valid , y_train , y_valid = train_test_split(x_train_full , y_train_full ,
                                                         test_size=0.15 ,random_state=42)

In [11]:
x_valid.shape

(2632, 8)

In [12]:
x_train = torch.FloatTensor(x_train)                #float tensor auto converts array into 32 bit float 
                                                    #dont need 64 bit float to preserve memory 
x_valid = torch.FloatTensor(x_valid)
x_test = torch.FloatTensor(x_test)
means = x_train.mean(dim=0 , keepdims=True)
stds = x_train.std(dim=0 , keepdims=True)
x_train = (x_train - means)/stds                    #standard scaler z score aka standard scaler
x_valid = (x_valid - means)/stds
x_test = (x_test - means)/stds



In [13]:
y_train = torch.FloatTensor(y_train).reshape(-1 , 1) 
y_valid = torch.FloatTensor(y_valid).reshape(-1 , 1)
y_test = torch.FloatTensor(y_test).reshape(-1 , 1)



In [14]:
torch.manual_seed(42)  #just like rng = numpy.random.default_rng
n_features = x_train.shape[1]
w = torch.rand((n_features , 1) , requires_grad=True) #random initial weights column matrix 
b = torch.tensor(0. , requires_grad=True) #bias
 

In [15]:
#linear reg

learning_rate = 0.42
n_epoch = 200

for epoch in range(n_epoch):
    y_pred = x_train @ w + b
    loss = ((y_pred - y_train) **2).mean()
    loss.backward()
    
    with torch.no_grad():
        b -= learning_rate * b.grad
        w -= learning_rate * w.grad 
        b.grad.zero_()
        w.grad.zero_()
    
    print(f"Epoch  : {epoch + 1}/{n_epoch} , loss: {loss.item()}")
    

Epoch  : 1/200 , loss: 7.675676345825195
Epoch  : 2/200 , loss: 1.4036966562271118
Epoch  : 3/200 , loss: 0.9067614674568176
Epoch  : 4/200 , loss: 0.7670203447341919
Epoch  : 5/200 , loss: 0.7075631618499756
Epoch  : 6/200 , loss: 0.6758161187171936
Epoch  : 7/200 , loss: 0.6550825834274292
Epoch  : 8/200 , loss: 0.6394436955451965
Epoch  : 9/200 , loss: 0.6266301274299622
Epoch  : 10/200 , loss: 0.6156783103942871
Epoch  : 11/200 , loss: 0.6061199903488159
Epoch  : 12/200 , loss: 0.5976894497871399
Epoch  : 13/200 , loss: 0.5902113318443298
Epoch  : 14/200 , loss: 0.583556592464447
Epoch  : 15/200 , loss: 0.5776218771934509
Epoch  : 16/200 , loss: 0.5723215341567993
Epoch  : 17/200 , loss: 0.5675822496414185
Epoch  : 18/200 , loss: 0.5633400082588196
Epoch  : 19/200 , loss: 0.5595393776893616
Epoch  : 20/200 , loss: 0.55613112449646
Epoch  : 21/200 , loss: 0.5530722141265869
Epoch  : 22/200 , loss: 0.5503243803977966
Epoch  : 23/200 , loss: 0.5478538870811462
Epoch  : 24/200 , loss: 

In [16]:
x_new = x_test[:3]
with torch.no_grad():
    y_pred = x_new @ w + b
    
y_pred

tensor([[0.7338],
        [1.7521],
        [2.6617]])

In [17]:
y_test[:3]

tensor([[0.4770],
        [0.4580],
        [5.0000]])

In [18]:
#with pytorch high level api
torch.manual_seed(42)
model = nn.Linear(in_features=n_features , out_features=1)



In [19]:
model.bias

Parameter containing:
tensor([0.3117], requires_grad=True)

In [20]:
model.weight

Parameter containing:
tensor([[ 0.2703,  0.2935, -0.0828,  0.3248, -0.0775,  0.0713, -0.1721,  0.2076]],
       requires_grad=True)